In [1]:
import torch
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

In [12]:
from load_data import *

In [13]:
X_tr, X_te, y_tr, y_te, xs_tr, xs_te = load_adult_data(svm=False,random_state=42)

for others min:  0 and max:  1


In [14]:
class NPsDataSet(TensorDataset):

    def __init__(self, *dataarrays):
        tensors = (torch.tensor(da).float() for da in dataarrays)
        super(NPsDataSet, self).__init__(*tensors)

In [15]:
y_tr = y_tr.astype('float32')
y_te = y_te.astype('float32')
train_data = NPsDataSet(X_tr, y_tr, xs_tr)
test_data = NPsDataSet(X_te, y_te, xs_te)

In [16]:
train_loader = DataLoader(train_data, batch_size=32, shuffle=True, drop_last=True)

In [17]:
print('# training samples:', len(train_data))
print('# batches:', len(train_loader))

# training samples: 31655
# batches: 989


# Training NN

In [18]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [19]:
class Classifier(nn.Module):

    def __init__(self, n_features, n_hidden=32, p_dropout=0.2):
        super(Classifier, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(n_features, n_hidden),
            nn.ReLU(),
            nn.Dropout(p_dropout),
            nn.Linear(n_hidden, n_hidden),
            nn.ReLU(),
            nn.Dropout(p_dropout),
            nn.Linear(n_hidden, n_hidden),
            nn.ReLU(),
            nn.Dropout(p_dropout),
            nn.Linear(n_hidden, 1),
        )

    def forward(self, x):
        return F.sigmoid(self.network(x))

In [20]:
n_features = train_data.tensors[0].shape[1]
clf = Classifier(n_features=n_features)
clf_criterion = nn.BCELoss()
clf_optimizer = optim.Adam(clf.parameters())

In [21]:
N_CLF_EPOCHS = 50

for epoch in range(N_CLF_EPOCHS):
    for x, y, _ in train_loader:
        clf.zero_grad()
        p_y = clf(x)
        loss = clf_criterion(p_y, y)
        loss.backward()
        clf_optimizer.step()

/home/srp/.pyenv/versions/pytorch15/lib/python3.8/site-packages/torch/nn/functional.py:1569: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/srp/.pyenv/versions/pytorch15/lib/python3.8/site-packages/torch/nn/modules/loss.py:516: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


# testing NN model

In [22]:
from fair_eval import calculate_prule_clf, calculate_odds_clf, calculate_parity_reg, calculate_group_loss,l2_loss,bce_loss, calculate_overall_accuracy

In [46]:
pred = (clf(torch.tensor(X_te).float())>0.5).float()
pred = pred.cpu().detach().numpy().flatten()

In [47]:
pred_ = clf(torch.tensor(X_te).float())
pred_ = pred_.cpu().detach().numpy().flatten()

In [41]:
calculate_prule_clf(pred,y_te,xs_te)

disparate impact:  32.06759723333161
disparate misclassification rate:  71.87386538883237
disparate false positive rate: nan
disparate false negative rate: nan


In [42]:
calculate_odds_clf(pred,y_te,xs_te)

equalized opportunity for 0.0 : 27.2104783730285
equalized opportunity for 1.0 : 82.72482656600694


In [57]:
calculate_group_loss(bce_loss,pred_,y_te,xs_te)

loss function:  bce_loss
loss value for group 0: -0.20911578834056854
loss value for group 1: -0.39355403184890747


In [37]:
calculate_overall_accuracy(pred,y_te)

0.8534679737598585

In [83]:
calculate_mistreatment(pred,y_te,xs_te)

[False False False ... False False False]


76.8922455350994

In [69]:
xs_te

array([[1.],
       [1.],
       [1.],
       ...,
       [0.],
       [1.],
       [1.]])